# Задание 1. Заливка и выделение границы.
* 1а. Рекурсивный алгоритм заливки на основе серий пикселов (линий)  заданным цветом.

* 1б. Рекурсивный алгоритм заливки на основе серий пикселов (линий)  рисунком из графического файла. Файл можно загрузить встроенными средствами и затем считывать точки изображения для использования в заливке. Рассмотреть случаи когда файл небольшого размера и заливается циклически и когда большой. Масштабировать не нужно.  Область рисуется мышкой. Область произвольной формы. Внутри могут быть отверстия. Точка, с которой начинается заливка, задается щелчком мыши. 

Клочкова

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton
from matplotlib.widgets import Button
from PIL import Image
import tkinter as tk
from tkinter import filedialog

%matplotlib tk

In [42]:
class FloodFillSystem:
    def __init__(self):
        self.canvas_size = (500, 800)
        self.reset_canvas()
        self.texture_img = None
        self.fill_mode = "color"
        self.drawing = False
        self.last_point = None
        self.line_color = [0, 0, 0]
        self.fill_color = [0.9, 0.2, 0.2]
        self.texture_start_point = None
        
    def reset_canvas(self):
        self.canvas = np.ones((self.canvas_size[0], self.canvas_size[1], 3), dtype=np.float32)
        
    def draw_line(self, x1, y1, x2, y2, thickness=3):
        points = self.get_line_points(x1, y1, x2, y2)
        for px, py in points:
            self.draw_thick_point(px, py, thickness)
            
    def get_line_points(self, x1, y1, x2, y2):
        points = []
        dx = abs(x2 - x1)
        dy = abs(y2 - y1)
        sx = 1 if x1 < x2 else -1
        sy = 1 if y1 < y2 else -1
        err = dx - dy
        
        while True:
            points.append((x1, y1))
            if x1 == x2 and y1 == y2:
                break
            e2 = 2 * err
            if e2 > -dy:
                err -= dy
                x1 += sx
            if e2 < dx:
                err += dx
                y1 += sy
        return points
    
    def draw_thick_point(self, x, y, thickness):
        for i in range(-thickness, thickness + 1):
            for j in range(-thickness, thickness + 1):
                px, py = x + i, y + j
                if (0 <= px < self.canvas_size[1] and 
                    0 <= py < self.canvas_size[0] and
                    i*i + j*j <= thickness*thickness):
                    self.canvas[py, px] = self.line_color

    def flood_fill(self, x, y):
        height, width = self.canvas.shape[:2]
        
        if x < 0 or x >= width or y < 0 or y >= height:
            return False
            
        target_color = self.canvas[y, x].copy()
        
        if self.fill_mode == "color":
            if np.array_equal(target_color, self.fill_color):
                return False
        else:
            if self.texture_img is None:
                print("Текстура не загружена!")
                return False
        
        if self.fill_mode == "texture":
            self.texture_start_point = (x, y)
        
        print(f"Начинаем заливку в точке ({x}, {y})...")
        
        if self.fill_mode == "color":
            self._flood_fill_solid(x, y, target_color, self.fill_color)
        else:
            self._flood_fill_texture(x, y, target_color)
            
        print("Заливка завершена!")
        return True

    def _flood_fill_solid(self, x, y, target_color, fill_color):
        height, width = self.canvas.shape[:2]
        
        if x < 0 or x >= width or y < 0 or y >= height:
            return
        
        if not np.array_equal(self.canvas[y, x], target_color):
            return
        
        x_left = x
        while x_left >= 0 and np.array_equal(self.canvas[y, x_left], target_color):
            x_left -= 1
        x_left += 1
        
        x_right = x
        while x_right < width and np.array_equal(self.canvas[y, x_right], target_color):
            x_right += 1
        x_right -= 1
        
        for i in range(x_left, x_right + 1):
            self.canvas[y, i] = fill_color
        
        for new_y in [y-1, y+1]:
            if 0 <= new_y < height:
                current_x = x_left
                while current_x <= x_right:
                    if np.array_equal(self.canvas[new_y, current_x], target_color):
                        self._flood_fill_solid(current_x, new_y, target_color, fill_color)
                        while (current_x <= x_right and 
                               np.array_equal(self.canvas[new_y, current_x], target_color)):
                            current_x += 1
                    else:
                        current_x += 1

    def _flood_fill_texture(self, x, y, target_color):
        height, width = self.canvas.shape[:2]
        
        if self.texture_img is None or self.texture_start_point is None:
            return
            
        tex_height, tex_width = self.texture_img.shape[:2]
        start_x, start_y = self.texture_start_point
        
        if x < 0 or x >= width or y < 0 or y >= height:
            return
        
        if not np.array_equal(self.canvas[y, x], target_color):
            return
        
        x_left = x
        while x_left >= 0 and np.array_equal(self.canvas[y, x_left], target_color):
            x_left -= 1
        x_left += 1
        
        x_right = x
        while x_right < width and np.array_equal(self.canvas[y, x_right], target_color):
            x_right += 1
        x_right -= 1
        
        for i in range(x_left, x_right + 1):
            tex_x = (i - start_x) % tex_width
            tex_y = (y - start_y) % tex_height
            texture_color = self.texture_img[tex_y, tex_x].copy()
            self.canvas[y, i] = texture_color
        
        for new_y in [y-1, y+1]:
            if 0 <= new_y < height:
                current_x = x_left
                while current_x <= x_right:
                    if np.array_equal(self.canvas[new_y, current_x], target_color):
                        self._flood_fill_texture(current_x, new_y, target_color)
                        while (current_x <= x_right and 
                               np.array_equal(self.canvas[new_y, current_x], target_color)):
                            current_x += 1
                    else:
                        current_x += 1

    def load_texture(self, filepath):
        try:
            img = Image.open(filepath)
            
            if img.mode != 'RGB':
                img = img.convert('RGB')
                
            img_array = np.array(img, dtype=np.float32) / 255.0
            
            print(f"Текстура загружена: {img_array.shape[1]}x{img_array.shape[0]}")
            
            self.texture_img = img_array
            return True
            
        except Exception as e:
            print(f"Ошибка загрузки текстуры: {e}")
            return False

fill_system = FloodFillSystem()

In [43]:
class DrawingApp:
    def __init__(self, fill_system):
        self.fill_system = fill_system
        self.fig = None
        self.ax = None
        self.im = None
        
    def start(self):
        self.fig = plt.figure(figsize=(14, 8))
        
        self.ax = plt.axes([0.03, 0.25, 0.94, 0.70]) 
        self.im = self.ax.imshow(self.fill_system.canvas)
        self.ax.set_title("СИСТЕМА ЗАЛИВКИ - Рисуйте левой кнопкой, заливайте правой", 
                         pad=15, fontsize=12, fontweight='bold')
        self.ax.axis('off')
        
        self.create_buttons()
        
        self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        self.fig.canvas.mpl_connect('button_release_event', self.on_release)
        self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
        
        self.info_text = self.fig.text(0.5, 0.15, 'Режим: ЗАЛИВКА ЦВЕТОМ', 
                                     ha='center', va='center', fontsize=11,
                                     bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue"))
        
        plt.show()
        
    def create_buttons(self):
        """Создает только необходимые кнопки"""
        button_height = 0.08
        button_width = 0.18
        start_y = 0.03
        
        self.ax_color = plt.axes([0.15, start_y, button_width, button_height])
        self.btn_color = Button(self.ax_color, 'Заливка цветом')
        self.btn_color.on_clicked(self.on_color_click)
        
        self.ax_texture = plt.axes([0.40, start_y, button_width, button_height])
        self.btn_texture = Button(self.ax_texture, 'Загрузить текстуру')
        self.btn_texture.on_clicked(self.on_load_texture_click)
        
        self.ax_reset = plt.axes([0.65, start_y, button_width, button_height])
        self.btn_reset = Button(self.ax_reset, 'Очистить холст')
        self.btn_reset.on_clicked(self.on_reset_click)
        
    def on_color_click(self, event):
        """Переключение на заливку цветом"""
        self.fill_system.fill_mode = "color"
        self.fill_system.fill_color = [0.9, 0.2, 0.2]  
        self.info_text.set_text('Режим: ЗАЛИВКА ЦВЕТОМ')
        self.info_text.set_bbox(dict(boxstyle="round,pad=0.5", facecolor="lightblue"))
        self.update_display()
        print("Режим: Заливка цветом")
        
    def on_load_texture_click(self, event):
        """Загрузка текстуры из файла"""
        print("Выберите файл текстуры...")
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(
            title="Выберите файл текстуры",
            filetypes=[("Image files", "*.png *.jpg *.jpeg *.bmp *.gif *.tiff")]
        )
        root.destroy()
        
        if file_path:
            if self.fill_system.load_texture(file_path):
                self.fill_system.fill_mode = "texture"
                tex_size = self.fill_system.texture_img.shape
                self.info_text.set_text(f'Режим: ЗАЛИВКА ТЕКСТУРОЙ ({tex_size[1]}x{tex_size[0]})')
                self.info_text.set_bbox(dict(boxstyle="round,pad=0.5", facecolor="lightgreen"))
                self.update_display()
        
    def on_reset_click(self, event):
        """Очистка холста"""
        self.fill_system.reset_canvas()
        self.update_display()
        print("Холст очищен!")
        
    def on_press(self, event):
        if event.inaxes != self.ax:
            return
            
        if event.button == MouseButton.LEFT:
            self.fill_system.drawing = True
            self.fill_system.last_point = (int(event.xdata), int(event.ydata))
            
        elif event.button == MouseButton.RIGHT:
            x, y = int(event.xdata), int(event.ydata)
            print(f"Заливка в точке: ({x}, {y})")
            
            success = self.fill_system.flood_fill(x, y)
            if success:
                self.update_display()
        
    def on_release(self, event):
        if event.button == MouseButton.LEFT:
            self.fill_system.drawing = False
            self.fill_system.last_point = None
            
    def on_motion(self, event):
        if (event.inaxes != self.ax or 
            not self.fill_system.drawing or 
            self.fill_system.last_point is None):
            return
            
        x, y = int(event.xdata), int(event.ydata)
        last_x, last_y = self.fill_system.last_point
        
        self.fill_system.draw_line(last_x, last_y, x, y, thickness=3)
        self.fill_system.last_point = (x, y)
        
        self.update_display()
        
    def update_display(self):
        self.im.set_data(self.fill_system.canvas)
        self.fig.canvas.draw()

app = DrawingApp(fill_system)

app.start()